In [304]:
import pandas as pd

In [305]:
credit = pd.read_csv('credit_data.csv')
user = pd.read_csv('users_data.csv')
txn = pd.read_csv('transactions_data.csv')
stlmt = pd.read_csv('settlements_data.csv')
cycles = pd.read_csv('cycles.csv')
fail = pd.read_csv('failure_events_data.csv')

In [306]:
user.columns
user.head(5)

,user_id,referrer,email,name,city_id
0,1,a,1@1.com,oh t,4.0
1,2,a,2@2.com,nk s,7.0
2,3,NaN,3@1.com,itna,NaN
3,4,a,4@3.com,osn,3.0
4,5,a,5@1.com,um s,7.0


In [307]:
#clean the user data, make referrer boolean, drop city NaN
user['referrer'].nunique()
user['referrer'].loc[~user['referrer'].isna()] = 1
user['referrer'].loc[user['referrer'].isna()] = 0
#user.dropna(subset=['city_id']);

In [308]:
#check for grouping in user, city
user.describe()
user.head(5)

,user_id,referrer,email,name,city_id
0,1,1,1@1.com,oh t,4.0
1,2,1,2@2.com,nk s,7.0
2,3,0,3@1.com,itna,NaN
3,4,1,4@3.com,osn,3.0
4,5,1,5@1.com,um s,7.0


In [309]:
#check for multi cities, group the user
user.groupby(['user_id', 'city_id']).filter(lambda x: len(x) > 1)

,user_id,referrer,email,name,city_id


In [310]:
credit.info()
credit.describe()
#credit is all clean

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18468 entries, 0 to 18467
Data columns (total 3 columns):
user_id                18468 non-null int64
cycle_id               18468 non-null int64
global_credit_limit    18468 non-null int64
dtypes: int64(3)
memory usage: 432.9 KB


,user_id,cycle_id,global_credit_limit
count,18468.000000,18468.000000,18468.000000
mean,8067.809129,14.217295,126512.887156
std,4362.935645,4.649221,65312.491560
min,1.000000,2.000000,50000.000000
25%,4284.750000,10.000000,75000.000000
50%,8436.500000,15.000000,150000.000000
75%,11720.250000,18.000000,150000.000000
max,16045.000000,22.000000,1000000.000000


##### IMP: Plot credit limit as y with cycle on the x, per user to see the trend.
This would be helpful in finding peculiar cases.

In [311]:
import matplotlib.pyplot as plt

In [312]:
c = credit.groupby(['user_id'],as_index=False)

In [313]:
c.head(5)

,user_id,cycle_id,global_credit_limit
0,2358,15,75000
1,2358,18,150000
2,10504,21,150000
3,10504,16,75000
4,9948,14,75000
5,9948,16,150000
6,7021,12,200000
7,3022,7,150000
8,15747,18,75000
9,3177,12,500000


In [314]:
c = credit.pivot(index='user_id', columns='cycle_id', values='global_credit_limit');
c.head(5)

cycle_id,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
user_id,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### The above observation shows that time series analysis cannot be done because of missing data


In [315]:
cycles.head(5) #cycles is all clean

,cycle_id,start_date,end_date
0,1,2015-11-01 00:00:00 +0000 +0000,2015-11-15 00:00:00 +0000 +0000
1,2,2015-11-16 00:00:00 +0000 +0000,2015-11-30 23:59:59 +0000 +0000
2,3,2015-12-01 00:00:00 +0000 +0000,2015-12-15 00:00:00 +0000 +0000
3,4,2015-12-16 00:00:00 +0000 +0000,2015-12-31 23:59:59 +0000 +0000
4,5,2016-01-01 00:00:00 +0000 +0000,2016-01-15 23:59:59 +0000 +0000


In [316]:
len(cycles)

23

#### The settlement data is very important
<ul><li>To understand the metric for good and bad behaviour, we need to look at the one time settlements and delayed settlements individually.
    <li>Metrics can be (onetime_amount/total_spend) and (delayed_amount/total_spend)*(-days_delayed)
</ul>

In [317]:
#First let's clean the settlement data; encode if neccessary
#encode the settlement_status
stlmt['settlement_status'] = stlmt.settlement_status.astype('category').cat.codes

In [318]:
#bill_pending = 0, bill_settled = 1, onetime = 2
#cycle id missing?
stlmt['cycle_id'].loc[stlmt['cycle_id'].isnull() == True]

28043   NaN
28060   NaN
Name: cycle_id, dtype: float64

In [319]:
#drop cycle_id NaN
stlmt.dropna(subset=['cycle_id']);
#remove days_delayed NaN
stlmt['days_delayed'].loc[stlmt['days_delayed'].isna()] = 0
#stlmt.head(5)
stlmt = stlmt.drop(columns=['id'])

In [320]:
stlmt.head(5)

,user_id,cycle_id,settlement_amount,settlement_status,days_delayed,bill_created_at,settled_at
0,1738,21.0,190500,2,0.000000,NaN,2016-09-15 15:29:45.245929 +0000 +0000
1,6055,20.0,138900,1,14.966468,2016-08-31 16:16:00.874006 +0000 +0000,2016-09-15 15:27:43.69227 +0000 +0000
2,10630,20.0,43100,1,14.985449,2016-08-31 15:47:52.453709 +0000 +0000,2016-09-15 15:26:55.238252 +0000 +0000
3,5766,21.0,75300,2,0.000000,NaN,2016-09-15 15:26:20.622842 +0000 +0000
4,12416,20.0,38000,1,14.971875,2016-08-31 16:05:29.841341 +0000 +0000,2016-09-15 15:24:59.843029 +0000 +0000


In [321]:
#we need total_spend_amount ; amount_settled_early ; amount_settled_late
#let's group by to see how the total spend amount comes out
f = {'settlement_amount':['sum']}
stlmt_total_spend = stlmt.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [322]:
stlmt_total_spend.columns = stlmt_total_spend.columns.droplevel(1)

In [323]:
stlmt_total_spend.columns

Index([u'user_id', u'cycle_id', u'settlement_amount'], dtype='object')

In [324]:
stlmt_total_spend = stlmt_total_spend.rename(index=str,columns = {'settlement_amount':'total_spend'})

In [325]:
stlmt_total_spend[60:70] #to check if sum for total spend is correct, ex.(12,8)

,user_id,cycle_id,total_spend
60,11,20.0,46700
61,12,8.0,163200
62,12,9.0,91500
63,12,10.0,232400
64,12,11.0,215500
65,12,12.0,52300
66,12,13.0,94000
67,12,14.0,80700
68,12,15.0,123300
69,12,16.0,289100


In [326]:
#first split the data into 3 -> 0, 1, 2 status

#### Onetime Amount

In [327]:
stlmt_onetime = stlmt.loc[stlmt['settlement_status']==2]

In [328]:
stlmt_onetime.head(5)

,user_id,cycle_id,settlement_amount,settlement_status,days_delayed,bill_created_at,settled_at
0,1738,21.0,190500,2,0.0,NaN,2016-09-15 15:29:45.245929 +0000 +0000
3,5766,21.0,75300,2,0.0,NaN,2016-09-15 15:26:20.622842 +0000 +0000
11,13439,21.0,23300,2,0.0,NaN,2016-09-15 15:11:54.071178 +0000 +0000
12,2875,21.0,31900,2,0.0,NaN,2016-09-15 15:10:08.857486 +0000 +0000
14,14269,21.0,56900,2,0.0,NaN,2016-09-15 15:09:11.348649 +0000 +0000


In [329]:
#select columns
stlmt_onetime = stlmt_onetime[['user_id','cycle_id','settlement_amount']];

In [330]:
stlmt_onetime.head(5)

,user_id,cycle_id,settlement_amount
0,1738,21.0,190500
3,5766,21.0,75300
11,13439,21.0,23300
12,2875,21.0,31900
14,14269,21.0,56900


In [331]:
#aggregate
f = {'settlement_amount':[sum]}
stlmt_onetime_agg = stlmt_onetime.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [332]:
stlmt_onetime_agg.columns = stlmt_onetime_agg.columns.droplevel(1)

In [333]:
stlmt_onetime_agg = stlmt_onetime_agg.rename(index=str,columns = {'settlement_amount':'onetime_amt'})

####  'Dealyed Amount'

In [334]:
stlmt_delayed = stlmt.loc[stlmt['settlement_status']==1]
stlmt_delayed.head(5)

,user_id,cycle_id,settlement_amount,settlement_status,days_delayed,bill_created_at,settled_at
1,6055,20.0,138900,1,14.966468,2016-08-31 16:16:00.874006 +0000 +0000,2016-09-15 15:27:43.69227 +0000 +0000
2,10630,20.0,43100,1,14.985449,2016-08-31 15:47:52.453709 +0000 +0000,2016-09-15 15:26:55.238252 +0000 +0000
4,12416,20.0,38000,1,14.971875,2016-08-31 16:05:29.841341 +0000 +0000,2016-09-15 15:24:59.843029 +0000 +0000
5,13371,20.0,14400,1,14.968118,2016-08-31 16:07:39.286497 +0000 +0000,2016-09-15 15:21:44.661884 +0000 +0000
6,13172,20.0,26400,1,14.953311,2016-08-31 16:27:01.547579 +0000 +0000,2016-09-15 15:19:47.578431 +0000 +0000


In [335]:
#select columns
stlmt_delayed = stlmt_delayed[['user_id','cycle_id','settlement_amount','days_delayed']];

In [336]:
#aggregate
f = {'settlement_amount':[sum],'days_delayed':[sum]}
stlmt_delayed_agg = stlmt_delayed.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [337]:
stlmt_delayed_agg.columns = stlmt_delayed_agg.columns.droplevel(1)

In [338]:
stlmt_delayed_agg = stlmt_delayed_agg.rename(index=str,columns = {'settlement_amount':'delayed_amt'})

In [339]:
stlmt_delayed_agg.head(5)

,user_id,cycle_id,delayed_amt,days_delayed
0,1,14.0,67200,0.023479
1,1,15.0,32500,0.044287
2,1,18.0,108600,0.010955
3,2,5.0,13000,10.377654
4,2,7.0,36500,0.235332


#### 'Pending Amount' 

In [340]:
stlmt_pending = stlmt.loc[stlmt['settlement_status']==0]
stlmt_pending.head(5)

,user_id,cycle_id,settlement_amount,settlement_status,days_delayed,bill_created_at,settled_at
2909,8848,20.0,59900,0,0.0,2016-08-31 16:49:42.545263 +0000 +0000,NaN
2910,12718,20.0,21400,0,0.0,2016-08-31 16:49:41.952401 +0000 +0000,NaN
2911,11267,20.0,54600,0,0.0,2016-08-31 16:49:29.127541 +0000 +0000,NaN
2912,12005,20.0,147400,0,0.0,2016-08-31 16:49:27.747393 +0000 +0000,NaN
2914,13781,20.0,8600,0,0.0,2016-08-31 16:49:27.409813 +0000 +0000,NaN


In [341]:
stlmt_pending = stlmt_pending[['user_id','cycle_id','settlement_amount']];

In [342]:
#aggregate
f = {'settlement_amount':[sum]}
stlmt_pending_agg = stlmt_pending.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [343]:
stlmt_pending_agg.columns = stlmt_pending_agg.columns.droplevel(1)

In [344]:
stlmt_pending_agg = stlmt_pending_agg.rename(index=str,columns = {'settlement_amount':'pending_amt'})

In [345]:
#add a column with boolean pending, later will fillna with 1 for others after join

In [346]:
stlmt_pending_agg['is_pending'] = 1

In [347]:
stlmt_pending_agg.head(5)

,user_id,cycle_id,pending_amt,is_pending
0,2,19.0,81100,1
1,7,15.0,11700,1
2,18,6.0,20000,1
3,21,17.0,57300,1
4,22,11.0,13000,1


#### For the user which has pending status, he will not have any future cycles (make pending as boolean, repayment future would be -inf with 0 credit limit)
It will be interesting to see the trend of somebody who has had all the 3 natures. (not possible because of missing data maybe)

In [348]:
print len(stlmt), len(stlmt_total_spend), len(stlmt_onetime), len(stlmt_delayed), len(stlmt_pending)
print len(stlmt), len(stlmt_total_spend), len(stlmt_onetime_agg), len(stlmt_delayed_agg), len(stlmt_pending_agg)

30683 28598 5703 22395 2585
30683 28598 5020 22395 2585


#### Now we must join all the types of spend per user, per cycle and fillna appropriately. 
Left Join would make sense?

In [349]:
#join first the total with delayed
stlmt_all = pd.merge(stlmt_total_spend,stlmt_delayed_agg, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [350]:
stlmt_all.head(5)

,user_id,cycle_id,total_spend,delayed_amt,days_delayed
0,1,14.0,67200,67200.0,0.023479
1,1,15.0,32500,32500.0,0.044287
2,1,18.0,108600,108600.0,0.010955
3,2,5.0,13000,13000.0,10.377654
4,2,7.0,36500,36500.0,0.235332


In [351]:
#join stlmt_all with onetime
stlmt_all = pd.merge(stlmt_all,stlmt_onetime_agg, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [352]:
stlmt_all.head(5)

,user_id,cycle_id,total_spend,delayed_amt,days_delayed,onetime_amt
0,1,14.0,67200,67200.0,0.023479,NaN
1,1,15.0,32500,32500.0,0.044287,NaN
2,1,18.0,108600,108600.0,0.010955,NaN
3,2,5.0,13000,13000.0,10.377654,NaN
4,2,7.0,36500,36500.0,0.235332,NaN


In [353]:
#join stlmt_all with pending
stlmt_all = pd.merge(stlmt_all,stlmt_pending_agg, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [354]:
stlmt_all.head(5)

,user_id,cycle_id,total_spend,delayed_amt,days_delayed,onetime_amt,pending_amt,is_pending
0,1,14.0,67200,67200.0,0.023479,NaN,NaN,NaN
1,1,15.0,32500,32500.0,0.044287,NaN,NaN,NaN
2,1,18.0,108600,108600.0,0.010955,NaN,NaN,NaN
3,2,5.0,13000,13000.0,10.377654,NaN,NaN,NaN
4,2,7.0,36500,36500.0,0.235332,NaN,NaN,NaN


In [355]:
len(stlmt_all)

28598

#### VVImp - Fillna with appropriate values 

In [356]:
#now create metrics, is_pending might be dummy as pending_amt is already there

In [357]:
#fraction for delayed, onetime, pending
stlmt_all['delayed_amt'] = (stlmt_all['delayed_amt']/stlmt_all['total_spend'])*stlmt_all['days_delayed']

In [358]:
stlmt_all['onetime_amt'] = stlmt_all['onetime_amt']/stlmt_all['total_spend']
stlmt_all['pending_amt'] = stlmt_all['pending_amt']/stlmt_all['total_spend']

In [359]:
stlmt_all = stlmt_all.fillna(0)

In [360]:
#drop irrelevant columns
stlmt_all = stlmt_all.drop(columns=['days_delayed'])#,'is_pending'])

In [361]:
stlmt_all[10:20]

,user_id,cycle_id,total_spend,delayed_amt,onetime_amt,pending_amt,is_pending
10,2,14.0,96600,13.601391,0.0,0.0,0.0
11,2,15.0,35000,0.646585,0.0,0.0,0.0
12,2,16.0,419949,13.564712,0.0,0.0,0.0
13,2,17.0,22000,2.571479,0.0,0.0,0.0
14,2,18.0,57630,0.982984,0.0,0.0,0.0
15,2,19.0,81100,0.000000,0.0,1.0,1.0
16,4,8.0,15000,0.000000,1.0,0.0,0.0
17,4,13.0,30000,3.586420,0.0,0.0,0.0
18,4,14.0,35000,0.774737,0.0,0.0,0.0
19,4,15.0,21000,10.970861,0.0,0.0,0.0


In [362]:
#when crossing stlmt_all with itself for the final vector, 
#ass there are many missing cycles, it would be interesting to cycle_id_x < cycle_id_y
#and from that keep only the closet pair ex. 13-21 13-16 13-18, keep 13-16

In [363]:
#join with transaction, credit and user data

###### modify the tables with timestamp fields to respective cycle, based on time range

In [364]:
#write a function that takes the data frane as input, the column name and appends the cycle_id
def getCycleId(df, colname):
    #change the colname type tp datetime
    df[colname] = pd.to_datetime(df[colname])
    #create a new col, cycle_id
    df['cycle_id'] = 0
    
    #now check for each cycle range and assign cycle_id
    for index, row in cycles.iterrows():
        st = row['start_date']
        end = row['end_date']
        cid = row['cycle_id']
        df['cycle_id'].loc[(df[colname] >= st) & (df[colname]<= end)] = cid   

In [365]:
getCycleId(txn,'created_at')
getCycleId(fail,'created_at')

In [366]:
#change amount to Rs.
fail['transaction_amount'] = fail['amount_in_paise']/100

### Let's visualize and look for patterns

##### There could be two main aspects to no repayment - 
1. Customers nature/ habit (Inherent)
2. Dissatisfaction (Induced)<br>
<b>Note:</b> Interesting would to be figure out when somebody forgets (nature not recurrent - False Positives)

In [367]:
fail_new = fail[['user_id','cycle_id', 'transaction_id', 'transaction_amount','failure_type']]
fail_new.head(5)
fail_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17993 entries, 0 to 17992
Data columns (total 5 columns):
user_id               17993 non-null int64
cycle_id              17993 non-null int64
transaction_id        17993 non-null int64
transaction_amount    17993 non-null float64
failure_type          17993 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 702.9+ KB


In [368]:
#aggregate
f = {'transaction_id':['count']}
fail_new  = fail_new.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [369]:
fail_new.head(5)

,user_id,cycle_id,transaction_id
,,,count
0,1,14,1
1,2,6,3
2,2,11,1
3,2,12,3
4,2,15,2


In [370]:
fail_new.columns = fail_new.columns.droplevel(1)

In [371]:
fail_new = fail_new.rename(index=str,columns = {'transaction_id':'num_failed_txns'})

In [372]:
fail_new.head(5)

,user_id,cycle_id,num_failed_txns
0,1,14,1
1,2,6,3
2,2,11,1
3,2,12,3
4,2,15,2


##### Let's work with transactions data
split first into latest and old(repayed transactions)

In [373]:
txn.head(5)

,transaction_id,user_id,merchant_id,transaction_amount,created_at,cycle_id
0,141695,13693,1,10100,2016-09-30 15:29:10.228672,22
1,141694,8166,1,20900,2016-09-30 15:29:00.286923,22
2,141688,15886,1,28100,2016-09-30 15:27:55.801021,22
3,141686,12014,8,18000,2016-09-30 15:27:17.891471,22
4,141684,12392,1,63500,2016-09-30 15:25:56.857979,22


In [374]:
txn_22 = txn[txn['cycle_id']==22]

In [375]:
txn_22.size

51996

In [376]:
txn.size

442386

In [377]:
txn_old = txn[txn['cycle_id']!=22]
txn_old = txn_old.drop(columns=['created_at']);
txn_old.head(5)

,transaction_id,user_id,merchant_id,transaction_amount,cycle_id
8666,125783,14496,1,13200,21
8667,125782,4054,1,19300,21
8668,125779,14501,1,19200,21
8669,125776,189,1,19800,21
8670,125775,6212,1,20400,21


In [378]:
txn_old.size
txn_old.merchant_id.unique()

array([ 1, 18, 10, 13,  7, 11,  6,  8, 14,  9, 15,  2,  3, 17, 16, 12,  4,
        5])

In [379]:
#tables of interest left -> user, txns, credit, settlements
user.columns

Index([u'user_id', u'referrer', u'email', u'name', u'city_id'], dtype='object')

In [380]:
user_rel = user.drop(columns=['email','name']);
user_rel.head(5)

,user_id,referrer,city_id
0,1,1,4.0
1,2,1,7.0
2,3,0,NaN
3,4,1,3.0
4,5,1,7.0


In [381]:
credit.head(5)

,user_id,cycle_id,global_credit_limit
0,2358,15,75000
1,2358,18,150000
2,10504,21,150000
3,10504,16,75000
4,9948,14,75000


In [382]:
user_rel.head(5) #we need to add user and credit data to the above dataframe, 
                 #then we will cross to get cycle = cycle+1

,user_id,referrer,city_id
0,1,1,4.0
1,2,1,7.0
2,3,0,NaN
3,4,1,3.0
4,5,1,7.0


In [383]:
credit.head(5)

,user_id,cycle_id,global_credit_limit
0,2358,15,75000
1,2358,18,150000
2,10504,21,150000
3,10504,16,75000
4,9948,14,75000


In [384]:
txn_old.head(10)

,transaction_id,user_id,merchant_id,transaction_amount,cycle_id
8666,125783,14496,1,13200,21
8667,125782,4054,1,19300,21
8668,125779,14501,1,19200,21
8669,125776,189,1,19800,21
8670,125775,6212,1,20400,21
8671,125774,14489,18,20079,21
8672,125771,13439,1,19000,21
8673,125769,3320,1,18200,21
8674,125768,12980,1,52800,21
8675,125767,3474,1,23900,21


In [385]:
#aggregate
f = {'transaction_id':['count']}
txn_old = txn_old.groupby(['user_id', 'cycle_id'],as_index=False).agg(f)

In [386]:
txn_old.columns = txn_old.columns.droplevel(1)
txn_old = txn_old.rename(index=str,columns = {'transaction_id':'num_sucsfl_txns'})

In [387]:
txn_old.head(5)

,user_id,cycle_id,num_sucsfl_txns
0,1,14,3
1,1,15,1
2,1,18,2
3,2,5,1
4,2,6,2


In [388]:
print len(stlmt_all), len(txn_old), len(credit), len(user_rel), len(fail_new)

28598 32981 18468 16050 11093


In [389]:
#join stlmt_all with txns
stlmt_all_new = pd.merge(stlmt_all,txn_old, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [390]:
stlmt_all_new.head(10)

,user_id,cycle_id,total_spend,delayed_amt,onetime_amt,pending_amt,is_pending,num_sucsfl_txns
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0
4,2,7,36500,0.235332,0.0,0.0,0.0,3.0
5,2,8,42200,0.779108,0.0,0.0,0.0,2.0
6,2,9,21200,0.742215,0.0,0.0,0.0,1.0
7,2,10,32900,5.651593,0.0,0.0,0.0,1.0
8,2,11,11700,6.647299,0.0,0.0,0.0,1.0
9,2,12,74300,8.467596,0.0,0.0,0.0,3.0


In [391]:
x = stlmt_all_new.num_sucsfl_txns.isna()

In [392]:
type(x)

pandas.core.series.Series

In [393]:
x = x[x!= False]
len(x) #these could indicate cycles with only failed txns are there, lets see

195

In [394]:
len(fail_new)

11093

In [395]:
#join stlmt_all with credit
stlmt_all_new = pd.merge(stlmt_all_new,credit, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [396]:
stlmt_all_new.head(10)

,user_id,cycle_id,total_spend,delayed_amt,onetime_amt,pending_amt,is_pending,num_sucsfl_txns,global_credit_limit
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0,75000.0
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0,150000.0
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0,NaN
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0,NaN
4,2,7,36500,0.235332,0.0,0.0,0.0,3.0,NaN
5,2,8,42200,0.779108,0.0,0.0,0.0,2.0,NaN
6,2,9,21200,0.742215,0.0,0.0,0.0,1.0,NaN
7,2,10,32900,5.651593,0.0,0.0,0.0,1.0,NaN
8,2,11,11700,6.647299,0.0,0.0,0.0,1.0,NaN
9,2,12,74300,8.467596,0.0,0.0,0.0,3.0,NaN


##### Strange that credit data is not available for so many users/cycles

In [397]:
#join stlmt_all with fail_new
stlmt_all_new = pd.merge(stlmt_all_new,fail_new, how='left', left_on=['user_id','cycle_id'], right_on = ['user_id','cycle_id'])

In [398]:
stlmt_all_new.head(10)

,user_id,cycle_id,total_spend,delayed_amt,onetime_amt,pending_amt,is_pending,num_sucsfl_txns,global_credit_limit,num_failed_txns
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0,75000.0,1.0
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0,150000.0,NaN
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0,NaN,NaN
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0,NaN,NaN
4,2,7,36500,0.235332,0.0,0.0,0.0,3.0,NaN,NaN
5,2,8,42200,0.779108,0.0,0.0,0.0,2.0,NaN,NaN
6,2,9,21200,0.742215,0.0,0.0,0.0,1.0,NaN,NaN
7,2,10,32900,5.651593,0.0,0.0,0.0,1.0,NaN,NaN
8,2,11,11700,6.647299,0.0,0.0,0.0,1.0,NaN,1.0
9,2,12,74300,8.467596,0.0,0.0,0.0,3.0,NaN,3.0


In [399]:
#join stlmt_all with user_rel
stlmt_all_new = pd.merge(stlmt_all_new,user_rel, how='left', left_on=['user_id'], right_on = ['user_id'])

In [400]:
stlmt_all_new.head(10)

,user_id,cycle_id,total_spend,delayed_amt,onetime_amt,pending_amt,is_pending,num_sucsfl_txns,global_credit_limit,num_failed_txns,referrer,city_id
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0,75000.0,1.0,1,4.0
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0,150000.0,NaN,1,4.0
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0,NaN,NaN,1,4.0
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0,NaN,NaN,1,7.0
4,2,7,36500,0.235332,0.0,0.0,0.0,3.0,NaN,NaN,1,7.0
5,2,8,42200,0.779108,0.0,0.0,0.0,2.0,NaN,NaN,1,7.0
6,2,9,21200,0.742215,0.0,0.0,0.0,1.0,NaN,NaN,1,7.0
7,2,10,32900,5.651593,0.0,0.0,0.0,1.0,NaN,NaN,1,7.0
8,2,11,11700,6.647299,0.0,0.0,0.0,1.0,NaN,1.0,1,7.0
9,2,12,74300,8.467596,0.0,0.0,0.0,3.0,NaN,3.0,1,7.0


In [401]:
len(stlmt_all_new)

28598

In [402]:
#rename the column headers to get better/more sense

In [403]:
names = {'onetime_amt':'advance_paymnt_score','delayed_amt':'late_payment_penalty'}
stlmt_all_new = stlmt_all_new.rename(index=str,columns = names )

In [404]:
stlmt_all_new.head(5)

,user_id,cycle_id,total_spend,late_payment_penalty,advance_paymnt_score,pending_amt,is_pending,num_sucsfl_txns,global_credit_limit,num_failed_txns,referrer,city_id
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0,75000.0,1.0,1,4.0
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0,150000.0,NaN,1,4.0
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0,NaN,NaN,1,4.0
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0,NaN,NaN,1,7.0
4,2,7,36500,0.235332,0.0,0.0,0.0,3.0,NaN,NaN,1,7.0


### Better would be to define a class for each behaviour
<ul><li>only_onetime - class 3
    <li>onetime_delayed - class 2
    <li>only_delayed - class 1
    <li>only_pending - class 0
</ul><br>
Then create a model that would predit the confidence value/probability that a user would belong to a particular class

In [405]:
#assign calss to each row
stlmt_all_new1 = stlmt_all_new
stlmt_all_new['class'] = -1

In [406]:
stlmt_all_new.head(4)

,user_id,cycle_id,total_spend,late_payment_penalty,advance_paymnt_score,pending_amt,is_pending,num_sucsfl_txns,global_credit_limit,num_failed_txns,referrer,city_id,class
0,1,14,67200,0.023479,0.0,0.0,0.0,3.0,75000.0,1.0,1,4.0,-1
1,1,15,32500,0.044287,0.0,0.0,0.0,1.0,150000.0,NaN,1,4.0,-1
2,1,18,108600,0.010955,0.0,0.0,0.0,2.0,NaN,NaN,1,4.0,-1
3,2,5,13000,10.377654,0.0,0.0,0.0,1.0,NaN,NaN,1,7.0,-1


In [407]:
stlmt_all_new['class'].loc[stlmt_all_new['is_pending'] == 1.0] = 0 #only pending class 0

In [408]:
stlmt_all_new['class'].loc[(stlmt_all_new['is_pending'] == 0) & (stlmt_all_new['late_payment_penalty'] > 0) & (stlmt_all_new['advance_paymnt_score'] == 0)] = 1 #only delayed class 1

In [409]:
stlmt_all_new['class'].loc[(stlmt_all_new['late_payment_penalty'] > 0) & (stlmt_all_new['advance_paymnt_score'] > 0)] = 2

In [410]:
stlmt_all_new['class'].loc[stlmt_all_new['class'] == -1] = 3 #only onetime class 3

In [411]:
stlmt_all_new = stlmt_all_new.drop(columns=['pending_amt','global_credit_limit'])

In [412]:
stlmt_all_new.head(5)

,user_id,cycle_id,total_spend,late_payment_penalty,advance_paymnt_score,is_pending,num_sucsfl_txns,num_failed_txns,referrer,city_id,class
0,1,14,67200,0.023479,0.0,0.0,3.0,1.0,1,4.0,1
1,1,15,32500,0.044287,0.0,0.0,1.0,NaN,1,4.0,1
2,1,18,108600,0.010955,0.0,0.0,2.0,NaN,1,4.0,1
3,2,5,13000,10.377654,0.0,0.0,1.0,NaN,1,7.0,1
4,2,7,36500,0.235332,0.0,0.0,3.0,NaN,1,7.0,1


#### Now we want to get the vector with user info, current cycle info and next cycle (this would be predition label) 
#### What would be better to see is - x = <last_class,  last_spend_data, last_txns, current_total_spend,current_txns> y = < next_class> 

#### if we had correct data, we could have cumulatively computer the measures for score and penalty, maybe

In [413]:
#only drop is_pending and reorder other columns
df_last_cycle = stlmt_all_new[['user_id','cycle_id','referrer','city_id','total_spend','advance_paymnt_score','late_payment_penalty','num_sucsfl_txns','num_failed_txns','class']]
names = {'user_id':'user_id_x', 'cycle_id':'cycle_id_x','total_spend':'lc_total_spend','num_sucsfl_txns':'lc_num_sucsfl_txns','num_failed_txns':'lc_num_failed_txns','class':'lc_class'}
df_last_cycle = df_last_cycle.rename(index=str, columns =names )

In [414]:
df_next_cycle = stlmt_all_new[['user_id','cycle_id','total_spend','num_sucsfl_txns','num_failed_txns','class']] #for next cycle we need, last cycles info and this cycles txns info
names = {'user_id':'user_id_y','cycle_id':'cycle_id_y','total_spend':'cc_total_spend','num_sucsfl_txns':'cc_num_sucsfl_txns','num_failed_txns':'cc_num_failed_txns','class':'cc_class'}
df_next_cycle = df_next_cycle.rename(index=str, columns =names )

In [415]:
df_last_cycle.head(5)

,user_id_x,cycle_id_x,referrer,city_id,lc_total_spend,advance_paymnt_score,late_payment_penalty,lc_num_sucsfl_txns,lc_num_failed_txns,lc_class
0,1,14,1,4.0,67200,0.0,0.023479,3.0,1.0,1
1,1,15,1,4.0,32500,0.0,0.044287,1.0,NaN,1
2,1,18,1,4.0,108600,0.0,0.010955,2.0,NaN,1
3,2,5,1,7.0,13000,0.0,10.377654,1.0,NaN,1
4,2,7,1,7.0,36500,0.0,0.235332,3.0,NaN,1


In [416]:
df_next_cycle.head(5)

,user_id_y,cycle_id_y,cc_total_spend,cc_num_sucsfl_txns,cc_num_failed_txns,cc_class
0,1,14,67200,3.0,1.0,1
1,1,15,32500,1.0,NaN,1
2,1,18,108600,2.0,NaN,1
3,2,5,13000,1.0,NaN,1
4,2,7,36500,3.0,NaN,1


In [417]:
df_final = pd.merge(df_last_cycle, df_next_cycle, how='outer', left_on=['user_id_x'], right_on=['user_id_y'])

In [418]:
df_final1 = df_final.loc[(df_final.user_id_x == df_final.user_id_y) & (df_final.cycle_id_x +1 == df_final.cycle_id_y)]

In [419]:
df_final1.head(10)

,user_id_x,cycle_id_x,referrer,city_id,lc_total_spend,advance_paymnt_score,late_payment_penalty,lc_num_sucsfl_txns,lc_num_failed_txns,lc_class,user_id_y,cycle_id_y,cc_total_spend,cc_num_sucsfl_txns,cc_num_failed_txns,cc_class
1,1,14,1,4.0,67200,0.0,0.023479,3.0,1.0,1,1,15,32500,1.0,NaN,1
24,2,7,1,7.0,36500,0.0,0.235332,3.0,NaN,1,2,8,42200,2.0,NaN,1
38,2,8,1,7.0,42200,0.0,0.779108,2.0,NaN,1,2,9,21200,1.0,NaN,1
52,2,9,1,7.0,21200,0.0,0.742215,1.0,NaN,1,2,10,32900,1.0,NaN,1
66,2,10,1,7.0,32900,0.0,5.651593,1.0,NaN,1,2,11,11700,1.0,1.0,1
80,2,11,1,7.0,11700,0.0,6.647299,1.0,1.0,1,2,12,74300,3.0,3.0,1
108,2,14,1,7.0,96600,0.0,13.601391,3.0,NaN,1,2,15,35000,1.0,2.0,1
122,2,15,1,7.0,35000,0.0,0.646585,1.0,2.0,1,2,16,419949,14.0,NaN,1
136,2,16,1,7.0,419949,0.0,13.564712,14.0,NaN,1,2,17,22000,1.0,2.0,1
150,2,17,1,7.0,22000,0.0,2.571479,1.0,2.0,1,2,18,57630,2.0,1.0,1


In [420]:
len(df_final1)
df_final1.columns

Index([u'user_id_x', u'cycle_id_x', u'referrer', u'city_id', u'lc_total_spend',
       u'advance_paymnt_score', u'late_payment_penalty', u'lc_num_sucsfl_txns',
       u'lc_num_failed_txns', u'lc_class', u'user_id_y', u'cycle_id_y',
       u'cc_total_spend', u'cc_num_sucsfl_txns', u'cc_num_failed_txns',
       u'cc_class'],
      dtype='object')

In [421]:
#drop descriptive cols, user and cycle id
df_final2 = df_final1.drop(columns=['user_id_x','cycle_id_x','user_id_y','cycle_id_y'])

In [422]:
cols = df_final2.columns.tolist()

In [423]:
cols

['referrer',
 'city_id',
 'lc_total_spend',
 'advance_paymnt_score',
 'late_payment_penalty',
 'lc_num_sucsfl_txns',
 'lc_num_failed_txns',
 'lc_class',
 'cc_total_spend',
 'cc_num_sucsfl_txns',
 'cc_num_failed_txns',
 'cc_class']

In [424]:
df_final2.head(10)

,referrer,city_id,lc_total_spend,advance_paymnt_score,late_payment_penalty,lc_num_sucsfl_txns,lc_num_failed_txns,lc_class,cc_total_spend,cc_num_sucsfl_txns,cc_num_failed_txns,cc_class
1,1,4.0,67200,0.0,0.023479,3.0,1.0,1,32500,1.0,NaN,1
24,1,7.0,36500,0.0,0.235332,3.0,NaN,1,42200,2.0,NaN,1
38,1,7.0,42200,0.0,0.779108,2.0,NaN,1,21200,1.0,NaN,1
52,1,7.0,21200,0.0,0.742215,1.0,NaN,1,32900,1.0,NaN,1
66,1,7.0,32900,0.0,5.651593,1.0,NaN,1,11700,1.0,1.0,1
80,1,7.0,11700,0.0,6.647299,1.0,1.0,1,74300,3.0,3.0,1
108,1,7.0,96600,0.0,13.601391,3.0,NaN,1,35000,1.0,2.0,1
122,1,7.0,35000,0.0,0.646585,1.0,2.0,1,419949,14.0,NaN,1
136,1,7.0,419949,0.0,13.564712,14.0,NaN,1,22000,1.0,2.0,1
150,1,7.0,22000,0.0,2.571479,1.0,2.0,1,57630,2.0,1.0,1


In [425]:
df_final_new = df_final2.fillna(0)

In [426]:
df_final_new.head(10)

,referrer,city_id,lc_total_spend,advance_paymnt_score,late_payment_penalty,lc_num_sucsfl_txns,lc_num_failed_txns,lc_class,cc_total_spend,cc_num_sucsfl_txns,cc_num_failed_txns,cc_class
1,1,4.0,67200,0.0,0.023479,3.0,1.0,1,32500,1.0,0.0,1
24,1,7.0,36500,0.0,0.235332,3.0,0.0,1,42200,2.0,0.0,1
38,1,7.0,42200,0.0,0.779108,2.0,0.0,1,21200,1.0,0.0,1
52,1,7.0,21200,0.0,0.742215,1.0,0.0,1,32900,1.0,0.0,1
66,1,7.0,32900,0.0,5.651593,1.0,0.0,1,11700,1.0,1.0,1
80,1,7.0,11700,0.0,6.647299,1.0,1.0,1,74300,3.0,3.0,1
108,1,7.0,96600,0.0,13.601391,3.0,0.0,1,35000,1.0,2.0,1
122,1,7.0,35000,0.0,0.646585,1.0,2.0,1,419949,14.0,0.0,1
136,1,7.0,419949,0.0,13.564712,14.0,0.0,1,22000,1.0,2.0,1
150,1,7.0,22000,0.0,2.571479,1.0,2.0,1,57630,2.0,1.0,1
